## 主觀進場 自動加碼

### 登入帳號

In [8]:
import shioaji as sj
api = sj.Shioaji()

import pandas as pd 
key = pd.read_csv('key.csv')
account = key.iloc[0,0]
password = key.iloc[0,1]
account = api.login(account,password)

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


### 登入模擬帳號

In [10]:
import shioaji as sj
import pandas as pd 
api = sj.Shioaji(simulation=True)
accounts = api.login("PAPIUSER06", "2222", contracts_timeout=10000)

Response Code: 0 | Event Code: 0 | Info: host '218.32.76.102:80', hostname '218.32.76.102:80' IP 218.32.76.102:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [ ]:
contract = api.Contracts.Futures.TXF.TXF202102
order = api.Order(action="Buy",
                price=10200,
                quantity=2,
                order_type="ROD",
                price_type="LMT",
                octype="Auto",
                account=api.futopt_account)
trade = api.place_order(contract, order)
trade

In [9]:
ticks = api.ticks(api.Contracts.Futures.MXF.MXF202102, "2021-01-08")
ticks_df = (pd.DataFrame({**ticks}))
print(ticks_df)
print(ticks_df['close'].tail(1))

      bid_volume  bid_price  ask_volume  ask_price                   ts  \
0              0        0.0           0        0.0  1609945200058000000   
1              2    14832.0           1    14847.0  1609945200608000000   
2              1    14833.0           1    14848.0  1609945201584000000   
3              1    14835.0           1    14853.0  1609945208519000000   
4              1    14848.0           1    14851.0  1609945216192000000   
...          ...        ...         ...        ...                  ...   
7483           4    15172.0           1    15174.0  1610039156609000000   
7484           4    15172.0           2    15175.0  1610039158138000000   
7485           4    15174.0           1    15176.0  1610039167236000000   
7486           4    15177.0           2    15179.0  1610039258880000000   
7487           4    15178.0           1    15180.0  1610039263341000000   

      volume    close  
0          4  14839.0  
1          1  14847.0  
2          1  14848.0  
3  

In [ ]:
# 主觀進場 自動加碼 大波段
import schedule
contract = api.Contracts.Futures.MXF.MXF202102
AvgCost =  14926

def sell():
    if now_price == S1:
        order = api.Order(action="Sell",
            price=now_price,
            quantity=1,
            order_type="ROD",
            price_type="LMT",
            octype="Auto",
            account=api.futopt_account)
        trade = api.place_order(contract, order)
        print('S1')
    if now_price == S2:
        order = api.Order(action="Sell",
            price=now_price,
            quantity=1,
            order_type="ROD",
            price_type="LMT",
            octype="Auto",
            account=api.futopt_account)
        trade = api.place_order(contract, order)
        print('S2')

def big_2():
    ticks = api.ticks(contract, "2021-01-08")
    ticks_df = (pd.DataFrame({**ticks}))['close']
    High = ticks_df.max()
    now_price = ticks_df.tail(1)
    # condition_1 : -80
    if (High - AvgCost ) <= 80:
        S1 = AvgCost - 10
        S2 = AvgCost - 70
        sell()
        print('condition_1:','now = ',now_price = ticks_df.tail(1),'High-AvgCost =',High - AvgCost,'Expected profit = -80')
    # condition_2 (B1+120) : +10
    if 130 >= (High - AvgCost ) > 80:
        S1 = AvgCost + 15
        S2 = AvgCost - 5 
        sell()
        print('condition_2:','now = ',now_price = ticks_df.tail(1),'High-AvgCost =',High - AvgCost,'Expected profit = +10')
    # condition_3 (B1+170) : +90
    if 180 >= (High - AvgCost ) > 130:
        S1 = AvgCost + 60
        S2 = AvgCost + 30 
        sell()
        print('condition_3:','now = ',now_price = ticks_df.tail(1),'High-AvgCost =',High - AvgCost,'Expected profit = +95')

    # condition_4 (B1+220) : >150
    if (High - AvgCost ) > 180:
        S1 = High - 90
        S2 = High - 120
        Expected_profit = (2*High-210)-2*AvgCost
        sell()
        print('condition_4:','now = ',now_price = ticks_df.tail(1),'High-AvgCost =',High - AvgCost,'Expected profit =',Expected_profit)

schedule.every(20).seconds.do(big_2)
while 1:
    schedule.run_pending()